# Collection

In [ ]:
import time                                 # Time module sleep function to add delays in execution
import pandas as pd                         # For creating Data Frame 
from selenium import webdriver              # To control and automate web browser actions
from selenium.webdriver.common.by import By # To locate elements on a web page
from selenium.webdriver.common.action_chains import ActionChains    # To perform advanced interactions [hover, click]
from selenium.webdriver.support.ui import WebDriverWait             # To wait for a certain condition to occur before proceeding
from selenium.webdriver.support import expected_conditions as EC    # To define the conditions to wait for in WebDriverWait

driver = webdriver.Chrome()         # which browser to control
actions = ActionChains(driver)      # where the actions should be used
driver.maximize_window()            # maximizes the web browser
url = 'https://www.redbus.in/'      # url to open
driver.get(url)                     # opens the url in browser
driver.implicitly_wait(5)           # wait time to load the page


def get_into_rtc_wise(driver):         # function to collect each rtc details links

    rtc_detail_link = driver.find_element(By.CLASS_NAME, 'rtcHeadViewAll').find_element(By.CSS_SELECTOR, 'a.OfferSection__ViewAllText-sc-16xojcc-1.eVcjqm').get_attribute('href')

    driver.get(rtc_detail_link)
        
    # Find all RTC details elements
    rtc_detail_link = driver.find_elements(By.CLASS_NAME, 'D113_ul_region_rtc')

    links_dict = {}
    
    # Loop through each RTC detail element to extract link text and href
    for rtc_link in rtc_detail_link:
        links = rtc_link.find_elements(By.TAG_NAME, 'a')
        for link in links:
            text = link.text
            href = link.get_attribute('href')
            if href:
                links_dict[text] = href

    # removing the rtc's which is not required
    remove_keys = ['TNSTC', 'PRTC', 'APSRTC', 'TSRTC', 'Bihar State Tourism Development Corporation (BSTDC)', 'WBTC (CTC)', 'West Bengal Transport Corporation', 'South Bengal State Transport Corporation (SBSTC)', 'BSRTC Operated By VIP Travels', 'Himachal Pradesh Tourism Development Corporation (HPTDC)', 'Sikkim Nationalised Transport (SNT)', 'Meghalaya Transport Corporation(MTC)', 'GSRTC']

    rtc_links_filtered = {key: value for key, value in links_dict.items() if not any(rtc in key for rtc in remove_keys)}

    # replacing the names of each RTC's
    replacements = {
        'Kadamba Transport Corporation Limited (KTCL)': 'KTCL',
        'KSRTC (Kerala)': 'KSRTC',
        'NORTH BENGAL STATE TRANSPORT CORPORATION': 'NBSTC',
        'Bihar State Road Transport Corporation (BSRTC)': 'BSRTC',
        'PEPSU (Punjab)': 'PEPSU',
        'RSRTC': 'RSRTC',
        'HRTC': 'HRTC',
        'Assam State Transport Corporation (ASTC)': 'ASTC',       
        'KAAC TRANSPORT': 'KAAC'
        }
    
    # Replace the full names with the shorter forms
    rtc_links_replaced = {replacements.get(key, key): value for key, value in rtc_links_filtered.items()}

    rtc_links = list(rtc_links_replaced.values())
    rtc_names = list(rtc_links_replaced.keys())

    return rtc_links, rtc_names


def get_bus_route_link_pagewise(driver):            # Function to collection Route wise links for each RTC

    all_route_links = []
    all_route_names = []

    # to loop through pages
    for page_number in range(1, 6):
        try:
            route_detail_page = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.route_details a.route")))
            
            # Extract links & Title
            for route in route_detail_page:
                Bus_Route_Link = route.get_attribute("href")
                Bus_Route_Name = route.get_attribute("title")
                all_route_links.append(Bus_Route_Link)
                all_route_names.append(Bus_Route_Name)
            
            pagination_container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[4]/div[12]')))
            
            next_page_button = pagination_container.find_element(By.XPATH, f'.//div[contains(@class,"DC_117_pageTabs") and text()="{page_number + 1}"]')
            
            actions = ActionChains(driver)
            actions.move_to_element(next_page_button).perform()         # Scrolls to the next page button
            time.sleep(2)
            next_page_button.click()        # Clicks on the next page button
        except:
            break                   # Break the loop if no more pages are found

    return all_route_links, all_route_names


def get_bus_details_route_wise(driver, route_name, route_link):     # Function to get bus details of each route
    
    # Date on which data to be extracted
    Date = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.ID, 'searchDat')))

    while Date.get_attribute('value') != '19 Jul':
            next_button = driver.find_element(By.CSS_SELECTOR, 'span.next')     # clicks on the 'next' webelemen to reach the reuired date
            next_button.click()
            time.sleep(1)

    # Click on each rtc segment to view buses
    rtc_buses = driver.find_elements(By.XPATH, '//i[contains(@class, "p-left-10") and contains(@class, "icon-down")]')
    for rtc in rtc_buses:
        rtc_buses = rtc.find_element(By.XPATH, '//i[contains(@class, "p-left-10") and contains(@class, "icon-down")]')
        actions.move_to_element(rtc_buses).click().perform()
        time.sleep(5)

    # Scroll down to the bottom of the page to load all buses - dynamic page 
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script("return document.body.scrollHeight")     # collects te height of the page after scrolling]
        if new_height == last_height:
            break
        last_height = new_height

    bus_elements = driver.find_elements(By.CSS_SELECTOR, "li.row-sec.clearfix")     # element under which all the required bus data is present
    bus_ids = [bus.get_attribute('id') for bus in bus_elements]
    driver.execute_script("window.scrollTo(0, 0);")                                 # 

    departure_date = driver.find_element(By.CSS_SELECTOR, "input.searchDate").get_attribute("value").strip().replace(' ', '-') + '-2024'

    time.sleep(2)

    bus_data = []

    # loop through each bus element & get the required details
    for bus_id in bus_ids:
        bus = driver.find_element(By.ID, bus_id)
        actions.move_to_element(bus).perform()          # Hovers to each bus element
        time.sleep(1)
        
        Bus_Route_Name = route_name
        Bus_Route_Link = route_link
        Bus_Route_From = route_name.split()[0]
        Bus_Route_To = route_name.split()[-1]
        Bus_Name = bus.find_element(By.CSS_SELECTOR, 'div.travels.lh-24.f-bold.d-color').text if bus.find_elements(By.CSS_SELECTOR, 'div.travels.lh-24.f-bold.d-color') else None
        Bus_Type = bus.find_element(By.CSS_SELECTOR, 'div.bus-type.f-12.m-top-16.l-color.evBus').text if bus.find_elements(By.CSS_SELECTOR, 'div.bus-type.f-12.m-top-16.l-color.evBus') else None
        Departure_Date = departure_date
        Departing_Time = bus.find_element(By.CSS_SELECTOR, 'div.dp-time.f-19.d-color.f-bold').text if bus.find_elements(By.CSS_SELECTOR, 'div.dp-time.f-19.d-color.f-bold') else None
        Reaching_Date = bus.find_element(By.CSS_SELECTOR, "div.next-day-dp-lbl").text.replace(' ', '-') + '-2024' if bus.find_elements(By.CSS_SELECTOR, "div.next-day-dp-lbl") else departure_date
        Reaching_Time = bus.find_element(By.CSS_SELECTOR, 'div.bp-time.f-19.d-color.disp-Inline').text if bus.find_elements(By.CSS_SELECTOR, 'div.bp-time.f-19.d-color.disp-Inline') else None
        Duration = bus.find_element(By.CSS_SELECTOR, 'div.dur.l-color.lh-24').text if bus.find_elements(By.CSS_SELECTOR, 'div.dur.l-color.lh-24') else None
        Availability = (bus.find_element(By.CSS_SELECTOR, 'div.seat-left.m-top-30').text.split()[0] if bus.find_elements(By.CSS_SELECTOR, 'div.seat-left.m-top-30') else (bus.find_element(By.CSS_SELECTOR, 'div.seat-left.m-top-16').text.split()[0] if bus.find_elements(By.CSS_SELECTOR, 'div.seat-left.m-top-16') else '0'))
        Star_Rating = bus.find_element(By.CSS_SELECTOR, 'div.rating-sec.lh-24 span').text if bus.find_elements(By.CSS_SELECTOR, 'div.rating-sec.lh-24 span') else '0'
        Price = bus.find_element(By.CSS_SELECTOR, 'span.f-19.f-bold').text if bus.find_elements(By.CSS_SELECTOR, 'span.f-19.f-bold') else '0'

        try:        # Collecting the Amenities info if available
            amenities_button = bus.find_element(By.XPATH, ".//li[contains(@class, 'amenties b-p-list')]/span[contains(text(), 'Amenities')]")
            actions.move_to_element(amenities_button).click().perform()
            time.sleep(2)
            amenities = bus.find_element(By.CLASS_NAME, "panels-container   ").text if bus.find_elements(By.CLASS_NAME, "panels-container   ") else None
            amenities = amenities.replace('\n', ', ')
        except Exception as e:
            amenities = None

        try:        # Collecting Boarding & Dropping Points if available
            Boarding_Dropping_Points_button = bus.find_element(By.XPATH, ".//li[contains(@class, 'amenties b-p-list')]/span[contains(text(), 'Boarding & Dropping Points')]")
            actions.move_to_element(Boarding_Dropping_Points_button).click().perform()
            time.sleep(2)
            Boarding_Points = bus.find_element(By.CLASS_NAME, "fl.w-50.bp_list_section").find_elements(By.TAG_NAME, 'span')
            titles = [points.get_attribute('title') for points in Boarding_Points if points.get_attribute('title') != '']
            Boarding_Points = ', '.join(titles)

            Dropping_Points = bus.find_element(By.CLASS_NAME, "fl.w-50.dp_list_section").find_elements(By.TAG_NAME, 'span')
            titles = [points.get_attribute('title') for points in Dropping_Points if points.get_attribute('title') != '']
            Dropping_Points = ', '.join(titles)

        except Exception as e:
            Boarding_Points = None
            Dropping_Points = None

        # Converting the collected data to Dictionary
        bus_details = {
            "Bus Route Name": Bus_Route_Name,
            "Bus Route Link": Bus_Route_Link,
            "Bus Route From": Bus_Route_From,
            "Bus Route To": Bus_Route_To,
            "Bus Name": Bus_Name,
            "Bus Type": Bus_Type,
            "Departure Time": f"{Departure_Date} {Departing_Time}",
            "Reaching Time": f"{Reaching_Date} {Reaching_Time}",
            "Duration": Duration,
            "Availability": Availability,
            "Star Rating": Star_Rating,
            "Price": Price,
            "Amenities": amenities,
            "Boarding Points": Boarding_Points,
            "Dropping Points": Dropping_Points
            }
        
        bus_data.append(bus_details)        # append the dictionary to list outside the loop
        
        time.sleep(2)

    df = pd.DataFrame(bus_data)             # converts the list to data frame
    return df

# Collects links & names of each RTC by using function defined above
rtc_links, rtc_names = get_into_rtc_wise(driver)        

# initiate empty list 
all_route_links = []
all_route_names = []

# loop through each rtc link to collect route related data
for rtc, rtc_name in zip(rtc_links, rtc_names):

    driver.get(rtc)     # opens each rtc by its link

    print(rtc_name)
    print(rtc)

    time.sleep(2)

    all_route_links, all_route_names = get_bus_route_link_pagewise(driver)      # Collects each route link

    zipped_list = list(zip(all_route_links, all_route_names))

    combined_df = pd.DataFrame()    # initiate empty dataframe

    # Loops through each route
    for route_link, route_name in zipped_list:
        
        driver.get(route_link)
        
        time.sleep(3)

        df = get_bus_details_route_wise(driver, route_name, route_link) 

        if df.empty:
            continue

        else:
            df.to_csv(f'{route_name}.csv', index=False)         # converts the dataframe collected to CSV
            combined_df = pd.concat([combined_df, df], ignore_index=True)   # Combines all the dataframe

    combined_df.to_csv(f'{rtc_name}.csv', index=False)      #  Converts to CSV


# EXTRACTION & CLEANING

In [1]:
# Importing required libraries 

import pandas as pd
import re

In [2]:
rtc_names = ['KTCL', 'KSRTC', 'NBSTC', 'BSRTC', 'PEPSU', 'RSRTC', 'HRTC', 'ASTC', 'KAAC', 'JKSRTC']

In [3]:
# Functions to clean the data collected

# sorts the data operator wise
def classify_bus(bus_name):
    for keyword in rtc_names:
        if keyword in bus_name:
            return 'Government'
    return 'Private'

# Sorts bus type as AC or Non AC
def classify_bus_type(bus_type):
    x = ['sleeper', 'multi', 'premium', 'benz', 'garuda']
    for i in x:
        if i in bus_type.lower():
            return 'Sleeper'
        else:
            return 'Seater'

# Sorts bus tyoe by sleeper or seater
def classify_ac_type(bus_type):
    x = ['ac', 'a/c', 'multi', 'sleeper']
    for i in x:
        if i in bus_type.lower() and 'non' not in bus_type.lower():
            return 'AC'
        else:
            return 'NON AC'

In [4]:
# Cleaning the data

all_rtc_df = pd.DataFrame()

for rtc_name in rtc_names:  
    df = pd.read_csv(f'{rtc_name}.csv')     # reads each rtc csv to dataframe
    df['RTC Name'] = rtc_name               # Creating a column to specify each rtc name
    df['Bus Route To'] = df['Bus Route Name'].apply(lambda x: re.sub(r'\s*\(.*?\)\s*', '', x).split()[-1])
    # converting to required data types
    df['Price'] = df['Price'].astype(float)
    df['Availability'] = df['Availability'].astype(int)
    df['Star Rating'] = df['Star Rating'].astype(float)
    df['Duration'] = df['Duration'].astype(str)
    df['Departure Time'] = pd.to_datetime(df['Departure Time'], format='%d-%b-%Y %H:%M')
    df['Reaching Time'] = pd.to_datetime(df['Reaching Time'], format='%d-%b-%Y %H:%M')

    # Adding additional columns
    df['Operator Type'] = df['Bus Name'].apply(classify_bus)
    df['Bus_Type1'] = df['Bus Type'].apply(classify_bus_type)
    df['Bus_Type2'] = df['Bus Type'].apply(classify_ac_type)

    
    all_rtc_df = pd.concat([all_rtc_df, df], ignore_index=True)         # combining all the df

all_rtc_df.to_csv('RedBus_Data.csv', index=False)                       # converting the combined df to CSV

# STORAGE

In [1]:
# Establishing connection to MySQL

import mysql.connector

connection = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "MySql_Password",
    database = "Redbus_Project"
)
cursor = connection.cursor()

In [10]:
# Creating table to insert the collected data

query = """
    CREATE TABLE IF NOT EXISTS bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Bus_Route_Name VARCHAR(255),
    Bus_Route_Link VARCHAR(255),
    Bus_Route_From VARCHAR(50),
    Bus_Route_To VARCHAR(50),
    Bus_Name VARCHAR(255),
    Bus_Type VARCHAR(100),
    Departing_Time DATETIME,
    Reaching_Time DATETIME,
    Duration VARCHAR(50),
    Availability INT,
    Star_Rating FLOAT,
    Price DECIMAL,
    Amenities TEXT,
    Boarding_Points TEXT,
    Reaching_Points TEXT,
    RTC_Name VARCHAR(10),
    Operator_Type VARCHAR(15),
    Bus_Type1 VARCHAR(10),
    Bus_Type2 VARCHAR(10)
);
"""
cursor.execute(query)

In [11]:
# Inserting the data

query = """insert into bus_routes
    (Bus_Route_Name, 
    Bus_Route_Link,
    Bus_Route_From,
    Bus_Route_To,
    Bus_Name,
    Bus_Type,
    Departing_Time,
    Reaching_Time,
    Duration,
    Availability,
    Star_Rating,
    Price,
    Amenities,
    Boarding_Points,
    Reaching_POints,
    RTC_Name,
    Operator_Type,
    Bus_Type1,
    Bus_Type2
    )
       values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
       
for index in all_rtc_df.index:
    row = all_rtc_df.loc[index].values
    val = [
    str(row[0]), str(row[1]), str(row[2]), str(row[3]), str(row[4]), str(row[5]),  
    row[6] if not pd.isna(row[6]) else None,  
    row[7] if not pd.isna(row[7]) else None,  
    row[8],  
    int(row[9]) if not pd.isna(row[9]) else 0, 
    float(row[10]) if not pd.isna(row[10]) else 0.0,  
    float(row[11]) if not pd.isna(row[11]) else 0.0,  
    str(row[12]), str(row[13]), str(row[14]), str(row[15]), str(row[16]), str(row[17]), str(row[18])]


    cursor.execute(query, val)
    connection.commit()